# Homework 6
任课老师：胡俊峰

负责助教：苏亚鲁、李浩然

注意：仅需要提交.ipynb文件，请**不要**将下发压缩包中的其他文件一并交上。

截止日期：3月18日24点

## 第零部分 代码理解
### 0.1 阅读代码预测结果，分析结果是否如预期，并按要求填充代码行，实现功能。

In [1]:
import pandas as pd
df = pd.DataFrame([[1, 2], [3, 4]], columns = ['a','b'])
print(df)
print(df['a']) 

   a  b
0  1  2
1  3  4
0    1
1    3
Name: a, dtype: int64


In [6]:
# 理解那个是0号索引
dft = df.T           
print(dft)
print(dft[1]['a'])  #  print(dft['a'][1] ?)
print(dft['a'][1])  #  KeyError, should be [column][row]

   0  1
a  1  3
b  2  4
3


KeyError: 'a'

In [13]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns = ['a','b'])
df3 = pd.concat([df,df2])  # 尝试用一行代码，df1，df2的横向拼接？
df4 = pd.concat([df,df2],axis=1)
print(df3)
print(df4)

   a  b
0  1  2
1  3  4
0  5  6
1  7  8
   a  b  a  b
0  1  2  5  6
1  3  4  7  8
['a', 'b']


In [21]:
# 请填充一行代码，实现打印 ‘b’列的值 <6 的df3的子视图
print(df3[df3['b'] < 6])

   a  b
0  1  2
1  3  4


In [22]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)

   A  B
0  4  9
1  4  9
2  4  9


dataframe.apply(function,axis)对一行或一列做出一些操作（axis=1遍历行，axis=0遍历列），可以使用匿名函数。

In [23]:
import numpy as np
print(df.apply(np.sum))
print(df.apply(np.sum, axis=1))

A    12
B    27
dtype: int64
0    13
1    13
2    13
dtype: int64


In [24]:
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [28]:
df.apply(lambda x: [1, 2], axis=1, result_type='expand')

,0,1
0,1,2
1,1,2
2,1,2


### 0.2 请认真阅读并运行以下示例代码，并思考C/S的多进程、多线程以及下面代码中select实现方式的优缺点。
注：文件夹下的example_client.py为本思考题对应的客户端文件。client.py为第一部分网络编程对应的客户端文件。（两个客户端均无需改动）

In [ ]:
import select
from socket import *
import sys
import queue

# 生成socket对象
server = socket(AF_INET, SOCK_STREAM)
server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
server.setblocking(False)

# 绑定地址，设置监听
server.bind(('localhost',9999))
server.listen(5)

# 将自己也放进待监测列表里
inputs = [server, ]
outputs = []
message_queues = {}

while True:

    rlist, wlist, elist = select.select(inputs,outputs,inputs) #如果没有任何fd就绪,那程序就会一直阻塞在这里

    for r in rlist:  # 遍历已经可以准备读取数据的 fd
        if r is server: # 如果这个 fd 是server，即 server 有数据待接收读取，说明有新的客户端连接过来了
            conn, client_addr = r.accept()
            print("new connection from",client_addr)
            conn.setblocking(False)
            inputs.append(conn) # 将这个新的客户端连接添加到检测的列表中
            message_queues[conn] = queue.Queue() # 用队列存储客户端发送来的数据，等待服务器统一返回数据

        else:          # 这个可读的 r 不是服务器，那就是某个客户端。就是说客户端发送数据过来了，这些数据处于待读取状态
            try:       # 异常处理，这是为了防止客户端异常断开报错（比如手动关掉客户端黑窗口，服务器也会跟着报错退出）
                data = r.recv(1024)
                if data:    # 根据判断data是否为空，判断客户端是否断开
                    print("收到来自[%s]的数据:" % r.getpeername()[0], data)
                    message_queues[r].put(data)   # 收到的数据先放到queue里,一会返回给客户端
                    if r not in outputs:
                        outputs.append(r)     # 放进可写的fd列表中，表明这些 fd 已经准备好去发送数据了。
                else:   # 如果数据为空，表明客户端断开了
                    print('客户端断开了')
                    if r in outputs:
                        outputs.remove(r)    #  清理已断开的连接
                    inputs.remove(r)         # 清理已断开的连接
                    del message_queues[r]    # 清理已断开的连接
            except ConnectionResetError:     # 如果报错，说明客户端断开了
                print("客户端异常断开了", r)
                if r in outputs:
                    outputs.remove(r)   # 清理已断开的连接
                inputs.remove(r)        # 清理已断开的连接
                del message_queues[r]  # 清理已断开的连接

    for w in wlist:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
        # 判断队列是否为空
        try :
            next_msg = message_queues[w].get_nowait()
        except queue.Empty:
            # print("client [%s]" % w.getpeername()[0], "queue is empty..")
            outputs.remove(w)
        # 队列不为空，就把队列中的数据改成大写，原样发回去
        else:
            # print("sending msg to [%s]"% w.getpeername()[0], next_msg)
            w.send(next_msg.upper())

    for e in elist:   # 处理报错的 fd
        e.close()
        print("Error occured in ",e.getpeername())
        inputs.remove(e)
        if e in outputs:
            outputs.remove(e)
        del message_queues[e]

new connection from ('127.0.0.1', 58241)
new connection from ('127.0.0.1', 58242)
收到来自[127.0.0.1]的数据: b'This is the message. '
收到来自[127.0.0.1]的数据: b'This is the message. '
收到来自[127.0.0.1]的数据: b'It will be sent '
收到来自[127.0.0.1]的数据: b'It will be sent '
收到来自[127.0.0.1]的数据: b'in parts.'
收到来自[127.0.0.1]的数据: b'in parts.'
客户端断开了
客户端断开了


## 第一部分：网络编程 (2.5分)

### 请将下列代码补充完整，从而实现如下功能：
**功能：**
* 为每个socket注册用户名，并检查用户名是否已经存在。若已经存在，则发送'Username already exists!'；否则发送'Welcome!'。(1.5分)
* 将注册成功的用户发送的信息，转发给其他的注册成功的用户。(1分)

**注：**
当补充完整代码后，可以运行client.py进行检验，其预期的输出为：\
connecting to localhost port 服务器端口 \
Welcome!to 第一个客户端端口 \
Username already exists!to 第二个客户端端口 \
Welcome!to 第二个客户端端口 \
Hello everybody!to 第二个客户端端口 

In [ ]:
from socket import *
from select import select
import queue

class Server:
    def __init__(self, host, port):
        self.server = socket(AF_INET, SOCK_STREAM)
        self.server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        self.server.setblocking(False)

        self.server.bind((host, port))
        self.server.listen(5)
        self.rlist = [self.server]
        self.wlist = []
        self.message_queues = {}
        self.users = {}  # 用于判断socket是否已经注册用户名

    def accept(self):
        while True:
            rs, ws, xs = select(self.rlist, self.wlist, self.rlist)
            for r in rs:
                if r is self.server:
                    # 服务器接受客户端连接
                    conn, addr = self.server.accept()
                    print("new connection from",addr)
                    conn.setblocking(False)
                    self.rlist.append(conn)
                    self.message_queues[conn] = queue.Queue()
                    self.users[conn] = False
                else:
                    try:
                        # 服务器接受客户端的消息并转发给所有客户端
                        data = r.recv(1024)
                        if data:
                            print("收到来自[%s]的数据:" % r.getpeername()[1], data.decode('utf-8'))
                            self.wlist.append(r)
                            if self.users[r]:  # 将已注册用户的信息广播给其他已注册用户
                                #print("%sregistered" % r.getpeername()[1])
                                for i in self.users:
                                    if i != r:
                                        self.wlist.append(i)
                                        self.message_queues[i].put(data)
                            else:  # 对未注册用户发送的昵称进行检验，并进行注册
                                #print("%s not registered" % r.getpeername()[1])
                                if data not in self.users.values():
                                    #print("new")
                                    self.users[r] = data
                                    self.message_queues[r].put("Welcome!".encode("utf-8"))
                                else:
                                    #print("used")
                                    self.message_queues[r].put("Username already exists!".encode("utf-8"))
                        else:
                            print('客户端断开了')
                            if r in self.wlist:
                                self.wlist.remove(r)   # 清理已断开的连接
                            self.rlist.remove(r)        # 清理已断开的连接
                            del self.message_queues[r]  # 清理已断开的连接
                    except ConnectionResetError:
                        print("客户端异常断开了", r)
                        if r in self.wlist:
                            self.wlist.remove(r)   # 清理已断开的连接
                        self.rlist.remove(r)        # 清理已断开的连接
                        del self.message_queues[r]  # 清理已断开的连接
            for w in ws:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
                # 判断队列是否为空
                try :
                    next_msg = self.message_queues[w].get_nowait()
                except queue.Empty:
                    # print("client [%s]" % w.getpeername()[0], "queue is empty..")
                    self.wlist.remove(w)
                # 队列不为空，就把队列中的数据改成大写，原样发回去
                else:
                    print("发送数据到 [%s]"% w.getpeername()[0], next_msg)
                    w.send(next_msg)

            for e in xs:   # 处理报错的 fd
                e.close()
                print("Error occured in ",e.getpeername())
                self.rlist.remove(e)
                if e in self.wlist:
                    self.wlist.remove(e)
                del self.message_queues[e] 

if __name__=='__main__':
    host = '0.0.0.0'      # 主机名
    port = 9999           # 端口号
    server = Server(host,port)
    server.accept()

new connection from ('127.0.0.1', 59123)
new connection from ('127.0.0.1', 59144)
收到来自[59123]的数据: Zhang San
收到来自[59144]的数据: Zhang San
发送数据到 [127.0.0.1] b'Welcome!'
发送数据到 [127.0.0.1] b'Username already exists!'
收到来自[59144]的数据: Xiao Ming
发送数据到 [127.0.0.1] b'Welcome!'
收到来自[59123]的数据: Hello everybody!
发送数据到 [127.0.0.1] b'Hello everybody!'
客户端断开了
客户端断开了


## 第二部分 Pandas基础练习（4.5分）

### 2.1 基本操作

In [3]:
import pandas as pd
import numpy as np

data = {
    'age': [24, 53, 23, 24, 33, 42, 57, 57, 36, 29, 53, 39],
    'gender': ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F'],
    'occupation': ['technician', 'other', 'writer', 'technician', 'technician', 'executive', 'administrator', 'administrator', 'administrator', 'student', 'lawyer', 'other'],
    'zip_code': ['85711', '94043', '32067', '43537', '15213', '98101', '91344', '91344', '05201', '01002', '90703', '30329']
}

创建一组从1开始递增的id，作为行索引，由data生成一个DataFrame。并以gender分组，打印每组的前两条。（0.5分）

In [34]:
# TODO
df1 = pd.DataFrame(data,index=range(1,13))
#print(df1)
#print([i for i in range(1,13) if (df1['gender'] == 'M')[i] == True])
print(df1[df1['gender'] == 'M'][:2])
print(df1[df1['gender'] == 'F'][:2])

   age gender  occupation zip_code
1   24      M  technician    85711
3   23      M      writer    32067
   age gender  occupation zip_code
2   53      F       other    94043
5   33      F  technician    15213


删除DataFrame中重复的行（保留一行），再删除所有occupation为"other"的行。（0.5分）

In [35]:
# TODO
df1 = df1.drop([i for i in df1.index if (df1['occupation'] == 'other')[i] == True])
df1 = df1.drop_duplicates(['zip_code'])

#print([i for i in range(1,13) if (df1['occupation'] == 'other')[i] == True])
print(df1)

    age gender     occupation zip_code
1    24      M     technician    85711
3    23      M         writer    32067
4    24      M     technician    43537
5    33      F     technician    15213
6    42      M      executive    98101
7    57      M  administrator    91344
9    36      M  administrator    05201
10   29      M        student    01002
11   53      M         lawyer    90703


以两种方式访问DataFrame的第一行。（0.5分）

In [51]:
# TODO
print(df1.loc[1:1])
print(df1.iloc[0:1])
print(df1[0:1])

print(df1.loc[1])
print(df1.iloc[0])

   age gender  occupation zip_code
1   24      M  technician    85711
   age gender  occupation zip_code
1   24      M  technician    85711
   age gender  occupation zip_code
1   24      M  technician    85711
age                   24
gender                 M
occupation    technician
zip_code           85711
Name: 1, dtype: object
age                   24
gender                 M
occupation    technician
zip_code           85711
Name: 1, dtype: object


以两种方式访问DataFrame的"occupation"列。（0.5分）

In [64]:
# TODO
print(df1.loc[:,"occupation":"occupation"],end='\n\n')
print(df1.iloc[:,2:3],end='\n\n')
print(df1["occupation"])

       occupation
1      technician
3          writer
4      technician
5      technician
6       executive
7   administrator
9   administrator
10        student
11         lawyer

       occupation
1      technician
3          writer
4      technician
5      technician
6       executive
7   administrator
9   administrator
10        student
11         lawyer

1        technician
3            writer
4        technician
5        technician
6         executive
7     administrator
9     administrator
10          student
11           lawyer
Name: occupation, dtype: object


打印出DataFrame中出现次数最多的职业名称（0.5分）

In [71]:
# TODO
m = max(res:=df1["occupation"].value_counts())
#print(m)
for i in df1["occupation"].unique():
    if res.loc[i] == m:
        #print(res)
        print(i)


technician


将data_2转换成DataFrame，与上一个DataFrame按照"zip_code"这一列合并，按两种合并方式：（0.5分）

1.合并后结果保留两个DataFrame的并集；

2.合并后只保留data_2的所有数据。

In [72]:
data_2 = {
    'zip_code': ['85711', '94043', '15213', '98101', '30329'],
    'name': ['Alan', 'Alice', 'Abigail', 'Bob', 'Dave']
}
df2 = pd.DataFrame(data_2)

# TODO
df3 = pd.merge(df1, df2, how="outer", on="zip_code")
df4 = pd.merge(df1, df2, how="right", on="zip_code")
print(df3)
print(df4) # 之前df1里删除的这里应该没有吧 94043和30329的数据已经在前面删掉了 不会扣分吧:O

     age gender     occupation zip_code     name
0   24.0      M     technician    85711     Alan
1   23.0      M         writer    32067      NaN
2   24.0      M     technician    43537      NaN
3   33.0      F     technician    15213  Abigail
4   42.0      M      executive    98101      Bob
5   57.0      M  administrator    91344      NaN
6   36.0      M  administrator    05201      NaN
7   29.0      M        student    01002      NaN
8   53.0      M         lawyer    90703      NaN
9    NaN    NaN            NaN    94043    Alice
10   NaN    NaN            NaN    30329     Dave
    age gender  occupation zip_code     name
0  24.0      M  technician    85711     Alan
1   NaN    NaN         NaN    94043    Alice
2  33.0      F  technician    15213  Abigail
3  42.0      M   executive    98101      Bob
4   NaN    NaN         NaN    30329     Dave


### 2.2 多级索引

In [123]:
# 构建多级索引DataFrame
# int数据由 np.random.randint(0, 150, size=(6, 3) 产生
# from tuples, from_arrays, from_product
import numpy as np
import pandas as pd

df3 = pd.DataFrame(np.random.randint(0, 150, size=(6, 3)),
                  columns=["Python", "C++", "Java"],
                  index=pd.MultiIndex.from_product([['张三', '李四', '王五'],
                                                  ['期中', '期末']]))
df3


Python  C++  Java
张三 期中     139  142    28
   期末     124   99   134
李四 期中     124   92   123
   期末     129   79    17
王五 期中      44    6   147
   期末     117   39    73

实现变换df3为三重索引的serial。(0.5分)

In [124]:
# TODO
df4 = df3.stack()
df4
#df4.index.names = ['name','period','course']
#df4.swaplevel('course','period')


张三  期中  Python    139
        C++       142
        Java       28
    期末  Python    124
        C++        99
        Java      134
李四  期中  Python    124
        C++        92
        Java      123
    期末  Python    129
        C++        79
        Java       17
王五  期中  Python     44
        C++         6
        Java      147
    期末  Python    117
        C++        39
        Java       73
dtype: int32

按期中40%，期末60%的比例，计算输出每位同学每门功课的总评成绩表。（0.5分）

In [148]:
# TODO
#df5 = pd.DataFrame({'Python':[0,0,0],'C++':[0,0,0],'Java':[0,0,0]}, index=['张三','李四','王五'])
data = {}
def calc(a,b):
    return a*0.4 + b*0.6
for i in df3.columns:
    for j in df3.index:
        data[j[0],i] = calc(*tuple(df3[i][j[0]]))
print(df5:=pd.Series(data).unstack())

      C++   Java  Python
张三  116.2   91.6   130.0
李四   84.2   59.4   127.0
王五   25.8  102.6    87.8


计算输出每门功课的最高分，最低分，以及平均分数表（课程名做行索引，列名用最高分，最低分，平均分数）。（0.5分）

In [155]:
# TODO
data = {}
for i in df5.columns:
    data[i] = [max(df5[i]), min(df5[i]), round(np.average(df5[i]),1)]
print(pd.DataFrame(data, index=["最高分","最低分","平均分数"]).T)

          最高分   最低分   平均分数
C++     116.2  25.8   75.4
Java    102.6  59.4   84.5
Python  130.0  87.8  114.9


## 第三部分 Pandas数据分析（3分）

使用Pandas的read_csv函数读取 IBM_2006-01-01_to_2018-01-01.csv 中的数据，得到一个DataFrame，通过shape和info()查看数据基本信息，并打印该数据的前10行观察一下。该数据列索引对应的分别是：日期，开盘价，最高价，最低价，收盘价，成交量，股票名称。（0.5分）

In [193]:
import pandas as pd

IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'
# TODO
df1 = pd.read_csv(IBM_file)
print(df1.shape)
print(df1.info)
print(df1[:][0:10])

(3020, 7)
<bound method DataFrame.info of             Date    Open    High     Low   Close    Volume Name
0     2006-01-03   82.45   82.55   80.81   82.06  11715200  IBM
1     2006-01-04   82.20   82.50   81.33   81.95   9840600  IBM
2     2006-01-05   81.40   82.90   81.00   82.50   7213500  IBM
3     2006-01-06   83.95   85.03   83.41   84.95   8197400  IBM
4     2006-01-09   84.10   84.25   83.38   83.73   6858200  IBM
...          ...     ...     ...     ...     ...       ...  ...
3015  2017-12-22  151.82  153.00  151.50  152.50   2990583  IBM
3016  2017-12-26  152.51  153.86  152.50  152.83   2479017  IBM
3017  2017-12-27  152.95  153.18  152.61  153.13   2149257  IBM
3018  2017-12-28  153.20  154.12  153.20  154.04   2687624  IBM
3019  2017-12-29  154.17  154.72  153.42  153.42   3327087  IBM

[3020 rows x 7 columns]>
         Date   Open   High    Low  Close    Volume Name
0  2006-01-03  82.45  82.55  80.81  82.06  11715200  IBM
1  2006-01-04  82.20  82.50  81.33  81.95   984060

观察到Open与Low这两列相比其他列都缺了一项，请删除DataFrame中所有不完整的数据。（0.5分）

In [194]:
# TODO
df1.drop(df1[df1["Open"].isna()].index) #No.2913 found

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM
...,...,...,...,...,...,...,...
3015,2017-12-22,151.82,153.00,151.50,152.50,2990583,IBM
3016,2017-12-26,152.51,153.86,152.50,152.83,2479017,IBM
3017,2017-12-27,152.95,153.18,152.61,153.13,2149257,IBM
3018,2017-12-28,153.20,154.12,153.20,154.04,2687624,IBM


由于股市周末和节假日不开盘，日期为2006-01-06的下一条数据就是2006-01-09，缺少了中间两天的数据。请将所有缺少的数据按照以下规则补充：成交量（volume）填充为零，开盘价等价格与前一天（最近的有数据的一天）保持相同。（0.5分）

提示：使用pandas.date_range生成时间序列

In [197]:
# TODO
df1Dates = list(df1['Date'])
dateRange = pd.date_range(start='2006-01-03', end='2018-01-01')
x = -1
for i in dateRange:
    x += 1
    if (d:=str(i)[:10]) not in df1Dates:
        df1 = pd.DataFrame(np.insert(df1.values, x, values=[d] + list(df1.iloc[x-1,1:5]) + [0] + [df1.iloc[x-1,6]], axis=0),columns=['Date','Open','High','Low','Close','Volume','Name'])
df1

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.2,82.5,81.33,81.95,9840600,IBM
2,2006-01-05,81.4,82.9,81.0,82.5,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-07,83.95,85.03,83.41,84.95,0,IBM
...,...,...,...,...,...,...,...
4377,2017-12-28,153.2,154.12,153.2,154.04,2687624,IBM
4378,2017-12-29,154.17,154.72,153.42,153.42,3327087,IBM
4379,2017-12-30,154.17,154.72,153.42,153.42,0,IBM
4380,2017-12-31,154.17,154.72,153.42,153.42,0,IBM


在DataFrame中按月份聚合，计算一年当中每个月的成交量总和，以及每个月收盘价的次高值。(1分)

提示：可以通过apply方法实现自定义的聚合函数。（有多个相同值的话，这里先简单地按照排完序取下标第二个就行了）

In [214]:
# TODO
sepa_head = [pd.date_range(start='2006-01-03', end='2006-01-03')] + list(pd.date_range(start='2006-01-03', end='2017-12-31',freq='MS'))
sepa_tail = list(pd.date_range(start='2006-01-03', end='2017-12-31',freq='M'))
dates = list(pd.date_range(start='2006-01-03', end='2017-12-31'))
data = {}
for i in range(len(sepa_head)):
    #print(dates.index(sepa_head[i]),dates.index(sepa_tail[i]))
    dfMonth = df1.loc[dates.index(sepa_head[i]):dates.index(sepa_tail[i]),'Volume']
    #print(sum(dfMonth))
    #print(list(dfMonth.sort_values())[-2])
    data[str(sepa_head[i])[:7]] = [sum(dfMonth), list(dfMonth.sort_values())[-2]]
print(pd.DataFrame(data, index=["成交量总和","收盘价次高值"]).T)

             成交量总和    收盘价次高值
Datetim  146611300  11032800
2006-02  111061200   9568600
2006-03  111811000   6414500
2006-04  114030800   8201800
2006-05  113238100   7019100
...            ...       ...
2017-08   80295410   4681545
2017-09   78281970   5447519
2017-10  121897687   9914169
2017-11   89776130   5676495
2017-12   88788538   6321801

[144 rows x 2 columns]


读取AMZN_2006-01-01_to_2018-01-01.csv，其中包含了Amazon的股票数据。
请以Date作为索引，'IBM'和'AMZN'作为列索引，构建一个Dataframe，其中的数据为两家公司每天的股票收盘价。（0.5分）

In [219]:
AMZN_file = 'AMZN_2006-01-01_to_2018-01-01.csv'
IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'

AMZN_data = pd.read_csv(AMZN_file)
IBM_data = pd.read_csv(IBM_file)

# TODO
AMZN_part = pd.DataFrame(AMZN_data, columns=['Date', 'Close'])
IBM_part = pd.DataFrame(IBM_data, columns=['Date', 'Close'])
df2 = pd.merge(IBM_part, AMZN_part, how='outer', on='Date')
df2 = df2.set_index('Date')
df2 = df2.set_axis(['IBM',"AMZN"], axis=1)
print(df2)

               IBM     AMZN
Date                       
2006-01-03   82.06    47.58
2006-01-04   81.95    47.25
2006-01-05   82.50    47.65
2006-01-06   84.95    47.87
2006-01-09   83.73    47.08
...            ...      ...
2017-12-22  152.50  1168.36
2017-12-26  152.83  1176.76
2017-12-27  153.13  1182.26
2017-12-28  154.04  1186.10
2017-12-29  153.42  1169.47

[3020 rows x 2 columns]
